<a href="https://colab.research.google.com/github/1ucky40nc3/stylegan3-super-recognizer/blob/dev/generate_faces_with_stylegan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Check if a GPU is connected to the runtime
!nvidia-smi

Thu Oct 21 18:56:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# @title Set up an environment (ONLY RUN ONCE)
!git clone https://github.com/NVlabs/stylegan3.git
!pip install ninja

%cd stylegan3/

downloaded_models = {}

In [ ]:
# @title Download a StyleGAN3 model
model = "stylegan3-t-ffhq-1024x1024.pkl" # @param ["stylegan3-t-ffhq-1024x1024.pkl", "tylegan3-t-ffhqu-1024x1024.pkl", "stylegan3-t-ffhqu-256x256.pkl", "stylegan3-r-ffhq-1024x1024.pkl", "stylegan3-r-ffhqu-1024x1024.pkl", "stylegan3-r-ffhqu-256x256.pkl", "stylegan3-t-metfaces-1024x1024.pkl", "stylegan3-t-metfacesu-1024x1024.pkl", "stylegan3-r-metfaces-1024x1024.pkl", "stylegan3-r-metfacesu-1024x1024.pkl", "stylegan3-t-afhqv2-512x512.pkl"]

if model not in downloaded_models:
    # Create the url to the resource.
    url = f"https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/{model}"
    print("[INFO] Downloading the model from NVIDIA.")

    # Download the pretrained model. 
    !wget $url

    # Mark the model as downloaded.
    downloaded_models[model] = url
else:
    print("[INFO] The model has already been downloaded.")

In [31]:
!python gen_images.py --outdir=out --trunc=1 --seeds=2 \
    --network=$model

Loading networks from "stylegan3-t-ffhq-1024x1024.pkl"...
Generating image for seed 2 (0/1) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.


In [32]:
!python gen_video.py --output=lerp.mp4 --trunc=1 --seeds=0-31 --grid=4x2 \
    --network=$model

Loading networks from "stylegan3-t-ffhq-1024x1024.pkl"...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
100% 480/480 [08:48<00:00,  1.10s/it]


In [70]:
from typing import Tuple
from typing import Optional

import torch
import numpy as np
from numpy.random import RandomState
from PIL import Image

from dnnlib.util import open_url
from legacy import load_network_pkl
from gen_images import make_transform

device = torch.device('cuda')

In [64]:
def generate(url: str,
             seed: int=2,
             translate: Tuple[float, float]=(0., 0.),
             rotate: float=0.,
             class_idx: Optional[int]=None,
             truncation_psi: float=1.,
             noise_mode: str='const') -> torch.Tensor:
    # Note: The first inference run may take longer because of plugins beeing set up.

    with open_url(url) as pkl:
        model = load_network_pkl(pkl)["G_ema"]
        model = model.to(device)

    # Generate the image seed.
    z = RandomState(seed).randn(1, model.z_dim)
    z = torch.from_numpy(z).to(device)

    # Compute an inverse rotation/translation matrix for the GAN.
    if hasattr(model.synthesis, 'input'):
        matrix = make_transform(translate, rotate)
        matrix = np.linalg.inv(matrix)
        matrix = torch.from_numpy(matrix)

        model.synthesis.input.transform.copy_(matrix)
    
    # Set class labels in the case for conditional networks.
    label = torch.zeros(1, model.c_dim, device=device)

    if model.c_dim != 0:
        assert class_idx, "The conditional network was trained with class labels and none was provided!"
        label[:, class_idx] = 1
    
    return model(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode).cpu()

In [67]:
def transform(img: torch.Tensor) -> torch.Tensor:
    # reshape from (batch, channels, height, width) to (batch, height, width, channels)
    img = img.permute(0, 2, 3, 1)
    
    # normalize the image from approximately [-1, 1] to [0, 255]
    img = img*127.5 + 128
    img = img.clamp(0, 255)

    return img.to(torch.uint8)

In [68]:
def save(img: torch.Tensor, filename: str="img.png"):
    img = img.numpy()
    Image.fromarray(img, 'RGB').save('img.png')

In [69]:
img = generate(model)
img = transform(img)
save(img[0].cpu())